##ML4 shilagel

#### 1. Загрузка данных и подключение библиотек

Подключение библиотек

In [7]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.4 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from category_encoders import CountEncoder

import warnings
warnings.filterwarnings('ignore')

Загрузка и чтение данных

In [11]:
df = pd.read_csv('data/training.csv')
df

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,73010,1,12/2/2009,ADESA,2001,8,MERCURY,SABLE,GS,4D SEDAN GS,...,4836.0,5937.0,NaN,NaN,18111,30212,GA,4200.0,0,993
72979,73011,0,12/2/2009,ADESA,2007,2,CHEVROLET,MALIBU 4C,LS,4D SEDAN LS,...,10151.0,11652.0,NaN,NaN,18881,30212,GA,6200.0,0,1038
72980,73012,0,12/2/2009,ADESA,2005,4,JEEP,GRAND CHEROKEE 2WD V,Lar,4D WAGON LAREDO,...,11831.0,14402.0,NaN,NaN,18111,30212,GA,8200.0,0,1893
72981,73013,0,12/2/2009,ADESA,2006,3,CHEVROLET,IMPALA,LS,4D SEDAN LS,...,10099.0,11228.0,NaN,NaN,18881,30212,GA,7000.0,0,1974


### 2. Разделение данных на training, validation, test

Преобразуем дату покупки

In [12]:
df['PurchDate'] = pd.to_datetime(df['PurchDate'])

Сортируем по дате

In [13]:
df = df.sort_values('PurchDate').reset_index(drop=True)
df

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,32389,0,2009-01-05,MANHEIM,2007,2,CHRYSLER,PACIFICA FWD 3.8L V6,Bas,4D SPORT,...,9906.0,11657.0,NaN,NaN,3453,80022,CO,6770.0,0,1389
1,32406,0,2009-01-05,MANHEIM,2005,4,FORD,FREESTAR FWD V6 3.9L,SES,4D PASSENGER 3.9L SES,...,5801.0,6949.0,NaN,NaN,22916,80022,CO,6160.0,0,941
2,32407,0,2009-01-05,MANHEIM,2004,5,DODGE,STRATUS 4C 2.4L I4 M,SE,4D SEDAN SE,...,4169.0,5114.0,NaN,NaN,3453,80022,CO,4250.0,0,1155
3,32408,0,2009-01-05,MANHEIM,2006,3,CHEVROLET,TRAILBLAZER EXT 4WD,LS,4D SUV 4.2L,...,10438.0,12158.0,NaN,NaN,22916,80022,CO,8180.0,0,1703
4,32409,0,2009-01-05,MANHEIM,2004,5,FORD,TAURUS 3.0L V6 EFI,SES,4D SEDAN SES DURATEC,...,4139.0,5351.0,NaN,NaN,22916,80022,CO,4900.0,0,825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,68095,0,2010-12-30,ADESA,2006,4,SUZUKI,AERIO,SX,4D WAGON SX,...,7601.0,8041.0,NaN,NaN,52598,28273,NC,4500.0,0,983
72979,68096,1,2010-12-30,ADESA,2006,4,SUZUKI,GRAND VITARA 2WD,NaN,4D SPORT UTILITY,...,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920
72980,68097,1,2010-12-30,ADESA,2005,5,FORD,TAURUS,SE,4D SEDAN SE,...,6476.0,9005.0,NO,GREEN,52598,28273,NC,4130.0,0,1053
72981,60722,1,2010-12-30,MANHEIM,2005,5,CHEVROLET,MALIBU 4C,Cla,4D SEDAN CLASSIC,...,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243


Разделяем данные

In [14]:
n = len(df)
train_end = int(n * 0.33)
valid_end = int(n * 0.66)

train_data = df.iloc[:train_end]
valid_data = df.iloc[:valid_end]
test_data = df.iloc[valid_end:]

### 3. Предварительная обработка данных

Выделяем категориальные и числовые колонки

In [15]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols = [c for c in cat_cols if c not in ['RefId', 'IsBadBuy', 'PurchDate']]  # исключаем ID, целевую и дату
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != 'IsBadBuy']

Разделение данных

In [16]:
X_train = train_data.drop(columns=['IsBadBuy'])
y_train = train_data['IsBadBuy']
X_valid = valid_data.drop(columns=['IsBadBuy'])
y_valid = valid_data['IsBadBuy']
X_test = test_data.drop(columns=['IsBadBuy'])
y_test = test_data['IsBadBuy']

Кодирование категориальных признаков

In [17]:
encoder = ce.CountEncoder(cols=cat_cols, handle_unknown='value', handle_missing='value')
X_train_cat = encoder.fit_transform(X_train[cat_cols])
X_valid_cat = encoder.transform(X_valid[cat_cols])
X_test_cat = encoder.transform(X_test[cat_cols])

Импутация и нормализация числовых признаков

In [18]:
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

Применяем к числовым признакам

In [19]:
X_train_num = imputer.fit_transform(X_train[num_cols])
X_valid_num = imputer.transform(X_valid[num_cols])
X_test_num = imputer.transform(X_test[num_cols])

X_train_num_scaled = scaler.fit_transform(X_train_num)
X_valid_num_scaled = scaler.transform(X_valid_num)
X_test_num_scaled = scaler.transform(X_test_num)

Объединение признаков

In [20]:
X_train_final = np.hstack([X_train_num_scaled, X_train_cat.values])
X_valid_final = np.hstack([X_valid_num_scaled, X_valid_cat.values])
X_test_final = np.hstack([X_test_num_scaled, X_test_cat.values])

Проверка на NaN

In [21]:
assert not np.isnan(X_train_final).any(), "Есть NaN в X_train_final!"
assert not np.isnan(X_valid_final).any(), "Есть NaN в X_valid_final!"
assert not np.isnan(X_test_final).any(), "Есть NaN в X_test_final!"

print("Форма X_train_final:", X_train_final.shape)
print("Пропусков нет — можно обучать модель!")

Форма X_train_final: (24084, 32)
Пропусков нет — можно обучать модель!


### 4. Обучение LogisticRegression, GaussianNB, KNN из sklearn

In [23]:
models = {
    'LogisticRegression': LogisticRegression(max_iter = 1000),
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(n_neighbors = 5)
}

results = {}
for name, model in models.items():
  model.fit(X_train_final, y_train)
  y_pred_proba = model.predict_proba(X_valid_final)[:, 1]
  auc = roc_auc_score(y_valid, y_pred_proba)
  gini = 2 * auc - 1
  results[name] = gini
  print(f"{name} - AUC: {auc:.4f}, Gini: {gini:.4f}")

LogisticRegression - AUC: 0.7374, Gini: 0.4749
GaussianNB - AUC: 0.7263, Gini: 0.4526
KNN - AUC: 0.7649, Gini: 0.5299


### 5. Реализация AUC ROC и Gini вручную

In [24]:
def compute_auc(y_true, y_score):
    # Сортируем по убыванию скоров
    indices = np.argsort(y_score)[::-1]
    y_true_sorted = y_true.iloc[indices] if hasattr(y_true, 'iloc') else y_true[indices]

    # Кумулятивные суммы
    pos = np.cumsum(y_true_sorted)
    total_pos = np.sum(y_true_sorted)
    neg = np.arange(len(y_true_sorted)) + 1 - pos
    total_neg = len(y_true_sorted) - total_pos

    # TPR и FPR
    tpr = np.concatenate([[0], pos / total_pos, [1]])
    fpr = np.concatenate([[0], neg / total_neg, [1]])

    # Площадь под кривой (трапеции)
    auc = 0
    for i in range(1, len(tpr)):
        auc += (fpr[i] - fpr[i-1]) * (tpr[i] + tpr[i-1]) / 2
    return auc

def gini_score(y_true, y_score):
    auc = compute_auc(y_true, y_score)
    return 2 * auc - 1

# Проверка
y_pred_lr = models['LogisticRegression'].predict_proba(X_valid_final)[:, 1]
our_gini = gini_score(y_valid, y_pred_lr)
sklearn_gini = 2 * roc_auc_score(y_valid, y_pred_lr) - 1

print(f"Наш Gini: {our_gini:.6f}")
print(f"sklearn Gini: {sklearn_gini:.6f}")
print(f"Разница: {abs(our_gini - sklearn_gini):.8f}")

Наш Gini: 0.474857
sklearn Gini: 0.474857
Разница: 0.00000000


### 6. Собственные версии классификаторов LogisticRegression, KNN и NaiveBayes

**Собственная Logistic Regression с SGD**

Линейная модель:

$$
z = \mathbf{w}^T \mathbf{x} + b = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b
$$

Функция, преобразующая линейную комбинацию в вероятность:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Логистическая регрессия:

$$
P(y=1 \mid \mathbf{x}) = \sigma(\mathbf{w}^T \mathbf{x} + b)
$$

In [25]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.001, epochs=1000, batch_size=64, tol=1e-4):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.tol = tol  # для ранней остановки
        self.w = None
        self.b = None
        self.losses_ = []  # можно отслеживать

    def _sigmoid(self, z):
        # Устойчивый sigmoid
        z = np.clip(z, -500, 500)
        return np.where(z >= 0,
                        1 / (1 + np.exp(-z)),
                        np.exp(z) / (1 + np.exp(z)))

    def _compute_loss(self, y_true, y_prob):
        # Лог-потеря с защитой от log(0)
        y_prob = np.clip(y_prob, 1e-15, 1 - 1e-15)
        return -np.mean(y_true * np.log(y_prob) + (1 - y_true) * np.log(1 - y_prob))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Инициализация весов (малые случайные значения)
        self.w = np.random.normal(0, 0.01, n_features)
        self.b = 0.0

        # Для отслеживания сходимости
        prev_loss = float('inf')

        for epoch in range(self.epochs):
            # Перемешиваем данные
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            epoch_loss = 0.0
            num_batches = 0

            for i in range(0, n_samples, self.batch_size):
                X_batch = X_shuffled[i:i + self.batch_size]
                y_batch = y_shuffled[i:i + self.batch_size]

                # Прямое распространение
                z = X_batch.dot(self.w) + self.b
                y_pred = self._sigmoid(z)

                # Вычисление градиентов
                m = len(y_batch)
                dw = (1 / m) * X_batch.T.dot(y_pred - y_batch)
                db = (1 / m) * np.sum(y_pred - y_batch)

                # Обрезаем градиенты (градиентный взрыв)
                dw = np.clip(dw, -1.0, 1.0)

                # Обновляем параметры
                self.w -= self.learning_rate * dw
                self.b -= self.learning_rate * db

                # Считаем лосс для отслеживания
                batch_loss = self._compute_loss(y_batch, y_pred)
                epoch_loss += batch_loss
                num_batches += 1

            epoch_loss /= num_batches
            self.losses_.append(epoch_loss)

            # Ранняя остановка
            if abs(prev_loss - epoch_loss) < self.tol:
                print(f"Converged at epoch {epoch}")
                break
            prev_loss = epoch_loss

            # Лог каждые 200 эпох
            # if epoch % 200 == 0:
            #     print(f"Epoch {epoch}, Loss: {epoch_loss:.4f}")

    def predict_proba(self, X):
        z = X.dot(self.w) + self.b
        prob_positive = self._sigmoid(z)
        prob_positive = np.clip(prob_positive, 1e-15, 1 - 1e-15)
        prob_negative = 1 - prob_positive
        # Возвращаем 2D массив: [P(0), P(1)]
        return np.column_stack([prob_negative, prob_positive])

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)

**Собственный KNN**

KNN — это непараметрический алгоритм, который классифицирует объект на основе схожести с другими объектами в обучающей выборке.

Основная идея:
Найти k ближайших к новому объекту x точек из обучающей выборки и присвоить ему класс, который чаще всего встречается среди них.

Расстояние между объектами (Евклидово):

$$
d(\mathbf{x}_i, \mathbf{x}_j) = \sqrt{\sum_{f=1}^{n} (x_{i,f} - x_{j,f})^2}
$$

Предсказание класса (голосование):

$$
\hat{y} = \text{mode}\left( \{ y_j \mid j \in \text{NN}_k(\mathbf{x}) \} \right)
$$

Взвешенное голосование (по расстоянию):

$$
P(y=c \mid \mathbf{x}) = \frac{\sum_{j \in \text{NN}_k(\mathbf{x})} w_j \cdot \mathbb{I}(y_j = c)}{\sum_{j \in \text{NN}_k(\mathbf{x})} w_j}
\quad \text{где} \quad
w_j = \frac{1}{d(\mathbf{x}, \mathbf{x}_j)}
$$

In [26]:
class MyKNN:
    def __init__(self, k=5):
        if k <= 0:
            raise ValueError("k must be positive.")
        self.k = k
        self.X_train = None
        self.y_train = None
        self.fitted_ = False

    def fit(self, X, y):
        # Преобразуем в numpy
        X = np.array(X)
        y = np.array(y).ravel()  # гарантируем 1D

        if X.shape[0] == 0:
            raise ValueError("Training set is empty.")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of samples.")

        self.X_train = X
        self.y_train = y
        self.fitted_ = True
        return self  # для Pipeline

    def predict_proba(self, X):
        prob1 = []  # P(y=1)
        for x in X:
            dists = np.linalg.norm(self.X_train - x, axis=1)
            k_indices = np.argsort(dists)[:self.k]
            prob1.append(np.mean(self.y_train[k_indices]))
        prob1 = np.array(prob1)
        prob0 = 1 - prob1
        return np.column_stack([prob0, prob1])  # форма: (n_samples, 2)

        return probs

    def predict(self, X):
        probas = self.predict_proba(X)
        return (probas >= 0.5).astype(int)

**Собственный Naive Bayes**

Наивный Байес — это вероятностный классификатор, основанный на теореме Байеса с предположением о независимости признаков.

Основная идея:
Найти класс c , который максимизирует апостериорную вероятность:

$$
\hat{y} = \underset{c \in C}{\text{argmax}} \; P(y = c \mid \mathbf{x})
$$

 Теорема Байеса

 $$
P(y = c \mid \mathbf{x}) = \frac{P(\mathbf{x} \mid y = c) \cdot P(y = c)}{P(\mathbf{x})}
$$

Наивное предположение (независимость признаков)

$$
P(\mathbf{x} \mid y = c) = \prod_{f=1}^{n} P(x_f \mid y = c)
$$

Апостериорная вероятность (упрощённая)

$$
\hat{y} = \underset{c}{\text{argmax}} \; P(y = c) \cdot \prod_{f=1}^{n} P(x_f \mid y = c)
$$

Для непрерывных признаков — Гауссов Наивный Байес

$$
P(x_f \mid y = c) = \frac{1}{\sqrt{2\pi\sigma_c^2}} \exp\left(-\frac{(x_f - \mu_c)^2}{2\sigma_c^2}\right)
$$

Для категориальных признаков — Multinomial / Categorical NB

$$
P(x_f = v \mid y = c) = \frac{N_{c,v} + \alpha}{N_c + \alpha \cdot V}
$$

In [27]:
class MyGaussianNB:
    def __init__(self):
        self.class_stats = {}
        self.priors = {}
        self.classes = None

    def _gaussian_log_pdf(self, x, mean, var):
        # Используем логарифм плотности — устойчивее
        var = np.clip(var, 1e-9, None)  # Защита от слишком маленькой дисперсии
        log_coeff = -0.5 * np.log(2 * np.pi * var)
        log_exp = -0.5 * (x - mean)**2 / var
        return log_coeff + log_exp  # log(P(x|class))

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_samples = X.shape[0]

        for c in self.classes:
            X_c = X[y == c]
            self.priors[c] = X_c.shape[0] / n_samples
            self.class_stats[c] = {
                'mean': np.mean(X_c, axis=0),
                'var': np.var(X_c, axis=0) + 1e-9  # Добавляем "шум"
            }

    def predict_proba(self, X):
        log_probs = []
        for c in self.classes:
            mean = self.class_stats[c]['mean']
            var = self.class_stats[c]['var']
            log_likelihood = np.sum(self._gaussian_log_pdf(X, mean, var), axis=1)
            log_prior = np.log(self.priors[c])
            log_probs.append(log_likelihood + log_prior)

        log_probs = np.stack(log_probs, axis=1)  # (n_samples, n_classes)
        log_probs -= log_probs.max(axis=1, keepdims=True)
        probs = np.exp(log_probs)
        probs /= probs.sum(axis=1, keepdims=True)  # (n_samples, n_classes)

        # Убедимся, что класс 1 — второй столбец
        if self.classes[0] == 1:  # если порядок [1, 0]
            probs = np.hstack([probs[:, 1:2], probs[:, 0:1]])  # меняем местами

        return probs  # всегда (n_samples, 2)

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)

Тестирование реализованных функций методов

In [48]:
from sklearn.metrics import roc_auc_score

models = {
    'LogisticRegression': MyLogisticRegression(learning_rate=0.001, epochs=1000, batch_size=64),
    'GaussianNB': MyGaussianNB(),
    'KNN': MyKNN(k=5)
}

results = {}
for name, model in models.items():
    print(f"Fitting {name}...")
    model.fit(X_train_final, y_train)

    print(f"Predicting {name}...")
    proba = model.predict_proba(X_valid_final)
    print(f"  predict_proba shape: {proba.shape}")

    if proba.ndim != 2 or proba.shape[1] != 2:
        print(f"{name}: Invalid shape {proba.shape}")
    else:
        y_pred_proba = proba[:, 1]

        if np.isnan(y_pred_proba).any():
            print(f"{name}: NaN values in predictions!")
        else:
            auc = roc_auc_score(y_valid, y_pred_proba)
            gini = gini_score(y_valid, y_pred_proba)
            results[name] = gini
            print(f"{name} - AUC: {auc:.4f}, Gini: {gini:.4f}")

Fitting LogisticRegression...
Converged at epoch 514
Predicting LogisticRegression...
  predict_proba shape: (48168, 2)
LogisticRegression - AUC: 0.5109, Gini: 0.0207
Fitting GaussianNB...
Predicting GaussianNB...
  predict_proba shape: (48168, 2)
GaussianNB - AUC: 0.7248, Gini: 0.4496
Fitting KNN...
Predicting KNN...
  predict_proba shape: (48168, 2)
KNN - AUC: 0.7649, Gini: 0.5535


### 7. Feature engineering

Создаём новые признаки

In [55]:
def create_features(df, df_train=None):
    df = df.copy()
    eps = 1e-8

    # 1. Дробные признаки
    if 'VehOdo' in df.columns and 'VehicleAge' in df.columns:
        df['MilesPerYear'] = df['VehOdo'] / (df['VehicleAge'] + 1)
        # Защита от inf
        df['MilesPerYear'] = df['MilesPerYear'].replace([np.inf, -np.inf], np.nan)

    if 'PurchPrice' in df.columns and 'MMRAcquisitionAuctionAveragePrice' in df.columns:
        df['PriceToMMR'] = df['PurchPrice'] / (df['MMRAcquisitionAuctionAveragePrice'] + eps)
        df['PriceToMMR'] = df['PriceToMMR'].replace([np.inf, -np.inf], np.nan)

    # 2. Групповые признаки
    if 'Make' in df.columns and 'VehOdo' in df.columns and df_train is not None:
        # Только по train
        mean_odo = df_train.groupby('Make')['VehOdo'].mean()
        overall_mean = mean_odo.mean()
        # Заполняем неизвестные марки — общим средним
        df['Make_Odo_Mean'] = df['Make'].map(mean_odo).fillna(overall_mean)

    # Можно добавить и другие:
    # if 'Auction' in df.columns and 'PurchPrice' in df.columns and df_train is not None:
    #     price_by_auction = df_train.groupby('Auction')['PurchPrice'].mean()
    #     df['Auction_PurchPrice_Mean'] = df['Auction'].map(price_by_auction).fillna(price_by_auction.mean())

    return df

Применяем к данным

In [61]:
X_train_enh = create_features(X_train, X_train)
X_valid_enh = create_features(X_valid, X_train)
X_test_enh = create_features(X_test, X_train)

Re-encode и масштабирование

In [62]:
enc = CountEncoder(cols=cat_cols, handle_unknown='value', handle_missing='value')

X_train_cat_enh = enc.fit_transform(X_train_enh[cat_cols])
X_valid_cat_enh = enc.transform(X_valid_enh[cat_cols])
X_test_cat_enh = enc.transform(X_test_enh[cat_cols])

X_train_num_enh = scaler.fit_transform(X_train_enh[num_cols])
X_valid_num_enh = scaler.transform(X_valid_enh[num_cols])
X_test_num_enh = scaler.transform(X_test_enh[num_cols])

X_train_final_enh = np.hstack([X_train_num_enh, X_train_cat_enh.values])
X_valid_final_enh = np.hstack([X_valid_num_enh, X_valid_cat_enh.values])
X_test_final_enh = np.hstack([X_test_num_enh, X_test_cat_enh.values])

Добавление импутации в пайплайн

In [63]:
imputer = SimpleImputer(strategy='median')
X_train_final_enh = imputer.fit_transform(X_train_final_enh)
X_valid_final_enh = imputer.transform(X_valid_final_enh)
X_test_final_enh = imputer.transform(X_test_final_enh)

Проверим, что NaN исчезли

In [64]:
print("X_train_final_enh has NaN:", np.isnan(X_train_final_enh).any())
print("X_valid_final_enh has NaN:", np.isnan(X_valid_final_enh).any())
print("X_test_final_enh has NaN:", np.isnan(X_test_final_enh).any())

X_train_final_enh has NaN: False
X_valid_final_enh has NaN: False
X_test_final_enh has NaN: False


Обучение модели на даныых после feature engineering

In [60]:
lr = MyLogisticRegression(learning_rate=0.001, epochs=1000, batch_size=64)
lr.fit(X_train_final_enh, y_train)
y_pred = lr.predict_proba(X_valid_final_enh)[:, 1]
gini_enh = gini_score(y_valid, y_pred)
print(f"Gini after feature engineering: {gini_enh:.4f}")

Converged at epoch 203
Gini after feature engineering: 0.1996


### 8. Определение и подбор подходящих признаков

С помощью коэффициентов логистичекской модели

In [35]:
# Обучим модель на ваших улучшенных данных
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_final_enh, y_train)

# Получим коэффициенты
coefficients = lr.coef_[0]  # shape: (n_features,)

In [36]:
# Создадим DataFrame для удобства
feature_names = [f"num_{i}" for i in range(X_train_num_enh.shape[1])] + \
                [f"cat_{i}" for i in range(X_train_cat_enh.shape[1])]

import pandas as pd
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coeff': np.abs(coefficients)
}).sort_values('abs_coeff', ascending=False)

print(coef_df.head(10))

   feature  coefficient  abs_coeff
3    num_3    -0.408034   0.408034
4    num_4     0.199537   0.199537
1    num_1    -0.185652   0.185652
2    num_2     0.185367   0.185367
13  num_13    -0.131913   0.131913
17  num_17     0.119908   0.119908
15  num_15    -0.101355   0.101355
0    num_0     0.096364   0.096364
14  num_14     0.050651   0.050651
11  num_11    -0.013723   0.013723


In [37]:
# Выберем топ-N признаков
n_top = 10
top_indices = coef_df.head(n_top).index

X_train_selected = X_train_final_enh[:, top_indices]
X_valid_selected = X_valid_final_enh[:, top_indices]

# Обучим и оценим
lr_selected = LogisticRegression(max_iter=1000)
lr_selected.fit(X_train_selected, y_train)
y_pred = lr_selected.predict_proba(X_valid_selected)[:, 1]
gini_manual = gini_score(y_valid, y_pred)

print(f"Gini после ручного отбора (top {n_top}): {gini_manual:.4f}")

Gini после ручного отбора (top 10): 0.3257


С помощью L1 - регуляризации

In [63]:
from sklearn.model_selection import GridSearchCV

# Создаём пайплайн
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Заполняем NaN
    ('scaler', StandardScaler()),                   # Масштабируем
    ('lr', LogisticRegression(
        penalty='l1',
        solver='saga',         # Только 'saga' поддерживает L1
        max_iter=3000          # Достаточно итераций
    ))
])

# Подбор C
param_grid = {
    'lr__C': [0.01, 0.1, 1, 10]  # C — сила регуляризации (чем меньше, тем сильнее)
}

# Поиск лучшего C по AUC
grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=0
)

# Обучаем на ваших улучшенных признаках
grid.fit(X_train_final_enh, y_train)

# Лучшая модель
best_model_l1 = grid.best_estimator_

print("Лучший параметр C:", grid.best_params_['lr__C'])
print("Лучший AUC на кросс-валидации:", grid.best_score_)

Лучший параметр C: 0.1
Лучший AUC на кросс-валидации: 0.7400129165737127


In [64]:
from sklearn.metrics import roc_auc_score

def gini_score(y_true, y_proba):
    auc = roc_auc_score(y_true, y_proba)
    return 2 * auc - 1

# Предсказания на valid
y_valid_pred_proba = best_model_l1.predict_proba(X_valid_final_enh)[:, 1]

# Gini
gini_l1 = gini_score(y_valid, y_valid_pred_proba)

print(f"\n🎯 Gini на валидации (L1): {gini_l1:.4f}")


🎯 Gini на валидации (L1): 0.4775


### 9. Применение нелинейных вариантов метода опорных векторов (Бонус)

Масштабируем (SVM чувствителен к масштабу)

In [89]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_valid_scaled = scaler.transform(X_valid_final)

Применяем SVC с RBF-ядром

In [94]:
svc = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=42)
svc.fit(X_train_scaled, y_train)
print("Оценка на train:", svc.score(X_train_scaled, y_train))

Оценка на train: 0.9064524165421026


In [96]:
imputer = SimpleImputer(strategy='median')
X_train_imp = imputer.fit_transform(X_train_final_enh)
X_valid_imp = imputer.transform(X_valid_final_enh)

# 2. Масштабирование ВСЕГО
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imp)
X_valid_scaled = scaler.transform(X_valid_imp)

# 3. Обучение SVC
svc = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', max_iter=10000, random_state=42)
svc.fit(X_train_scaled, y_train)

# 4. Проверка классов
print("Классы:", svc.classes_)

# 5. Предсказание
y_valid_proba = svc.predict_proba(X_valid_scaled)[:, 1]
gini_svc = 2 * roc_auc_score(y_valid, y_valid_proba) - 1
print(f"Исправленный SVC Gini: {gini_svc:.4f}")

Классы: [0 1]
Исправленный SVC Gini: 0.5532


### 10. Выбор лучшей модели и настройка гипперпараметоров

Лучшая модель:

LogisticRegression + улучшенные признаки (fractions + groupby) + CountEncoder + масштабирование + импутация

In [97]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(max_iter=10000))
])

Настройка через GridSearchCV

In [98]:
param_grid = {
    'lr__C': [0.01, 0.1, 1, 10, 100],
    'lr__penalty': ['l1', 'l2'],
    'lr__solver': ['saga']  # saga поддерживает и l1, и l2
}

# Или для elasticnet:
# param_grid = {
#     'lr__C': [0.1, 1, 10],
#     'lr__penalty': ['elasticnet'],
#     'lr__l1_ratio': [0.1, 0.5, 0.9],
#     'lr__solver': ['saga']
# }

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=0
)

# Обучение
grid.fit(X_train_final_enh, y_train)

print("Лучшие параметры:", grid.best_params_)
print("Лучший AUC (CV):", grid.best_score_)

Лучшие параметры: {'lr__C': 0.01, 'lr__penalty': 'l2', 'lr__solver': 'saga'}
Лучший AUC (CV): 0.7400750911512759


Оценка Gini на валидации

In [99]:
def gini_score(y_true, y_proba):
    return 2 * roc_auc_score(y_true, y_proba) - 1

best_model = grid.best_estimator_
y_valid_proba = best_model.predict_proba(X_valid_final_enh)[:, 1]
gini_valid = gini_score(y_valid, y_valid_proba)

print(f"🎯 Final Gini (valid): {gini_valid:.4f}")

🎯 Final Gini (valid): 0.4782


### 11. Сравнение показателей Gini для всех трех наборов данных

In [100]:
# Предсказания на всех выборках
y_train_proba = best_model.predict_proba(X_train_final_enh)[:, 1]
y_valid_proba = best_model.predict_proba(X_valid_final_enh)[:, 1]
y_test_proba = best_model.predict_proba(X_test_final_enh)[:, 1]

# Gini
gini_train = gini_score(y_train, y_train_proba)
gini_valid = gini_score(y_valid, y_valid_proba)
gini_test = gini_score(y_test, y_test_proba)

print(f"🎯 Training Gini:   {gini_train:.4f}")
print(f"🎯 Validation Gini: {gini_valid:.4f}")
print(f"🎯 Test Gini:       {gini_test:.4f}")

🎯 Training Gini:   0.4982
🎯 Validation Gini: 0.4782
🎯 Test Gini:       0.3047


### 12. Реализация расчета показателей Recall, Precision, F1 score, AUC PR

In [83]:
def precision_recall_f1_aucpr(y_true, y_proba, threshold=0.5):
    # Бинаризуем предсказания
    y_pred = (y_proba >= threshold).astype(int)

    # Стандартные метрики
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # AUC PR = Average Precision
    auc_pr = average_precision_score(y_true, y_proba)

    return precision, recall, f1, auc_pr

Обучим модели и сравним на test

In [87]:
models = {
    'LogisticRegression': LogisticRegression(max_iter = 1000),
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(n_neighbors = 5)
}

# Обучение
for name, model in models.items():
    model.fit(X_train_final_enh, y_train)

# Оценка на test
print("=== Test Metrics (AUC PR) ===")
results = {}
for name, model in models.items():
    # Предсказания
    y_proba = model.predict_proba(X_test_final_enh)[:, 1]

    # Метрики
    prec, rec, f1, auc_pr = precision_recall_f1_aucpr(y_test, y_proba)
    results[name] = {'Precision': prec, 'Recall': rec, 'F1': f1, 'AUC PR': auc_pr}

    print(f"{name} - Prec: {prec:.3f}, Rec: {rec:.3f}, F1: {f1:.3f}, AUC PR: {auc_pr:.3f}")

=== Test Metrics (AUC PR) ===
LogisticRegression - Prec: 0.812, Rec: 0.226, F1: 0.354, AUC PR: 0.442
GaussianNB - Prec: 0.056, Rec: 0.064, F1: 0.060, AUC PR: 0.164
KNN - Prec: 0.376, Rec: 0.232, F1: 0.287, AUC PR: 0.249


### 13. Лучший метрический показатель для "lemon" cars

Лучшая hard-label метрика:
**Recall**

Цель бизнеса: минимизировать убытки от покупки "lemon"

False Negative (FN): купили плохую машину → огромные убытки
False Positive (FP): отклонили хорошую машину → потеря сделки, но не катастрофа.
Мы гораздо больше боимся пропустить "lemon", чем случайно отклонить хороший авто.